<a href="https://colab.research.google.com/github/smf-9000/kaggle/blob/main/%5B004_2%5D_ann_tps_sep_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Additional feature extraction from row before the last from ann model-classifier

In [ ]:
%config Completer.use_jedi = False

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow, imread

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture

import scipy.stats as stats

import lightgbm as lgb
import warnings

import optuna

In [ ]:
R_SEED = 37

In [ ]:
train_data_b = pd.read_csv('/kaggle/input/tpssep20210041/train_data_b.csv')
test_data_b = pd.read_csv('/kaggle/input/tpssep20210041/test_data_b.csv')
target_data = pd.read_csv('/kaggle/input/tpssep20210041/target_data.csv')
submit_data = pd.read_csv('/kaggle/input/tpssep20210041/submit_data.csv')

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, Model

In [ ]:
from sklearn.metrics import roc_auc_score

def auroc(y_true, y_pred):
    return tf.py_function(roc_auc_score, (y_true, y_pred), tf.double)

In [ ]:
def ANN(input_shape=236):
    
    model = models.Sequential(
        [
            layers.Input(shape=input_shape),
            layers.Dense(256, activation='selu'),
            layers.BatchNormalization(),
            layers.Dropout(rate=0.4),
            layers.Dense(128, activation='selu'),
            layers.BatchNormalization(),
            layers.Dropout(rate=0.2),
            layers.Dense(16, activation='selu', name='f_e'),
            layers.Dense(1, activation='sigmoid')
        ],
        name='TPS-SEP21'
    )
    
    model.compile(
                    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.00002),
                    loss = 'binary_crossentropy',
                    metrics = ['accuracy'])
    return model

In [ ]:
model = ANN()
model.summary()

Model: "TPS-SEP21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               60672     
_________________________________________________________________
batch_normalization (BatchNo (None, 256)               1024      
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
batch_normalization_1 (Batch (None, 128)               512       
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
f_e (Dense)                  (None, 16)                20

In [ ]:
history = model.fit(
    train_data_b,
    target_data,
    epochs = 20,
    shuffle = True,
#     validation_split = 0.4,
    batch_size = 128,)

Epoch 1/20
7484/7484 [==============================] - 45s 6ms/step - loss: 0.7993 - accuracy: 0.5413
Epoch 2/20
7484/7484 [==============================] - 43s 6ms/step - loss: 0.6337 - accuracy: 0.6573
Epoch 3/20
7484/7484 [==============================] - 43s 6ms/step - loss: 0.5893 - accuracy: 0.7098
Epoch 4/20
7484/7484 [==============================] - 43s 6ms/step - loss: 0.5647 - accuracy: 0.7343
Epoch 5/20
7484/7484 [==============================] - 45s 6ms/step - loss: 0.5517 - accuracy: 0.7456
Epoch 6/20
7484/7484 [==============================] - 44s 6ms/step - loss: 0.5455 - accuracy: 0.7504
Epoch 7/20
7484/7484 [==============================] - 42s 6ms/step - loss: 0.5397 - accuracy: 0.7551
Epoch 8/20
7484/7484 [==============================] - 41s 5ms/step - loss: 0.5359 - accuracy: 0.7570
Epoch 9/20
7484/7484 [==============================] - 40s 5ms/step - loss: 0.5342 - accuracy: 0.7581
Epoch 10/20
7484/7484 [==============================] - 40s 5ms/step - l

In [ ]:
# model.save("nn_model.h5")
# model = models.load_model("../input/004-2-ann-tps-sep-2021/nn_model.h5")

In [ ]:
# output = model(test_data_b.values)
# submit_data['claim'] = np.squeeze(np.array(output))
# submit_data.to_csv('submission_ann.csv', index=False)
# submit_data

In [ ]:
fe_idx = [l.name for l in model.layers].index('f_e')
feature_extractor = Model(
                inputs=model.inputs,
                outputs=model.layers[fe_idx].output)

In [ ]:
col = ['nn_f' + str(i) for i in range(model.layers[fe_idx].output_shape[1])]

In [ ]:
col

['nn_f0',
 'nn_f1',
 'nn_f2',
 'nn_f3',
 'nn_f4',
 'nn_f5',
 'nn_f6',
 'nn_f7',
 'nn_f8',
 'nn_f9',
 'nn_f10',
 'nn_f11',
 'nn_f12',
 'nn_f13',
 'nn_f14',
 'nn_f15']

In [ ]:
new_features_train = feature_extractor(train_data_b.values)
new_features_test = feature_extractor(test_data_b.values)

In [ ]:
train_data_new_features = pd.DataFrame(data=np.array(new_features_train), index=train_data_b.index, columns=col)

In [ ]:
test_data_new_features = pd.DataFrame(data=np.array(new_features_test), index=test_data_b.index, columns=col)

In [ ]:
train_data_new_features.head()

,nn_f0,nn_f1,nn_f2,nn_f3,nn_f4,nn_f5,nn_f6,nn_f7,nn_f8,nn_f9,nn_f10,nn_f11,nn_f12,nn_f13,nn_f14,nn_f15
0,-1.210514,0.532672,0.437113,0.872868,0.732253,0.344190,0.607266,0.396993,0.248075,1.288122,-0.196867,-0.148372,0.496609,0.217578,0.294164,0.896073
1,0.322745,1.598228,-1.016876,-0.396802,0.017066,0.027303,0.525136,-0.143549,-0.105835,4.477329,0.067328,0.427172,0.755869,0.058427,1.457199,6.357563
2,0.166408,-1.340501,1.317596,1.464931,1.063987,1.542492,-0.251860,0.628439,0.315774,-1.747668,1.406090,0.576749,-0.646716,0.805818,-1.354134,-1.757971
3,-1.495363,-0.640164,1.407812,0.496428,0.107750,0.191731,-0.640228,0.798172,-0.627507,-1.408519,-0.375085,0.325995,0.409651,0.458943,-1.261604,-1.685391
4,2.280773,-1.430678,1.089380,0.775491,0.651635,3.028939,0.870203,0.547089,-0.290916,-1.757262,1.393477,1.437148,-0.886092,-0.144421,-1.432159,-1.758097


In [ ]:
test_data_new_features.head()

,nn_f0,nn_f1,nn_f2,nn_f3,nn_f4,nn_f5,nn_f6,nn_f7,nn_f8,nn_f9,nn_f10,nn_f11,nn_f12,nn_f13,nn_f14,nn_f15
0,-1.108520,-0.064040,0.914978,0.169181,-0.627552,-0.489166,0.423882,0.005536,0.098328,0.161746,-0.397844,0.225627,0.461899,0.295720,-0.592789,0.992889
1,-0.647702,1.613844,-1.159137,0.085489,0.377783,-0.034059,0.195312,0.062766,0.456793,3.482244,0.316348,0.357182,0.072802,-0.526592,0.773572,6.601974
2,-0.866087,0.604774,0.217579,-0.293156,-0.520723,-0.045758,0.181501,-0.469932,0.559400,1.621467,-0.879474,0.296437,0.318895,0.249893,0.212182,1.268957
3,0.545873,2.017678,-0.757720,-0.202564,0.358158,-0.204031,0.461005,-0.569477,-0.286507,3.053555,-0.176965,0.614605,0.224506,0.013721,1.285336,6.382481
4,0.026905,1.575511,-1.390320,-0.382868,0.343403,-0.653048,0.209651,0.366772,1.167329,2.902498,-0.605426,0.129572,0.135938,-0.005438,1.113387,6.323200


In [ ]:
train_data_new_features.to_csv('train_data_nn_features.csv', index=False)
test_data_new_features.to_csv('test_data_nn_features.csv', index=False)